Dataset preparation for pmcl = '764327358/01'

In [23]:
import json
import csv
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import cycleFetch as cf
import random
from csv import DictWriter as dw
from csv import DictReader as dr
import csv 
import numpy as np
from cycle import *

In [56]:
# prepare metadata variables for later processing
# prepare a metadata dictionary for faster metadata access

metadata = ''
metadata_dict = {}
with open('full_data.json', 'r') as f:
    metadata = list(json.load(f))[1]
assert metadata['pmcl'] == '764327358/01', 'PMCL number incorrect'
devices = metadata['devices']
count = 0
l = []
for device in devices:
    cycles = device['cycle_ids']
    for i, cycle in enumerate(cycles):
        
        assert cycle['cycleId'] not in metadata_dict, 'Cycle duplicate found'
        id = str(int(cycle['cycleId']))
        d = {id: (cycle['starttime'], cycle['endtime'], cycle['duration'], 0)}
        if ('FSR Required' in cycle):
            d[id] = (cycle['starttime'], cycle['endtime'], cycle['duration'], 1) # set label
            count += 1
            l.append(id)
        metadata_dict.update(d)

print(l)
print(len(l))

['14554342', '14554351', '14554360', '14554368', '14554384', '14554408', '14554457', '14554458', '14554453', '14554469', '14554475', '14554502', '14554508', '14554524', '14554530', '14554548', '14554553', '14554579', '14554587', '14555564', '14650435', '14650892', '14652039', '14924290', '14925133', '14925828', '14928847', '14929766', '14931154', '14942392', '14943525', '14944579', '14946969', '14947867', '14951808', '14968788', '14925057', '14925231', '14935227', '14935557', '14942453', '14942628', '14943015', '14944081', '14944896', '14946975', '14947747', '14948679', '14949366', '14949671', '15011180', '15050624', '15056908']
53


In [ ]:
# get sample cycles
# for pmcl = 764327358/01 first

cycle_ids = []
for device in devices:
    # device_counter -= 1
    device_id = device['deviceId']
    # print(f"Now executing device {device_id}")
    cycles = device['cycle_ids']
    # cycle_counter = 50
    # while (cycle_counter) > 0:
    for cycle in cycles:
        # cycle_counter -= 1
        # while (cycle_counter > 0):
            # cycle_counter -= 1
        cycle_id = cycle['cycleId']
        cycle_ids.append(cycle_id)

# cycle_ids = random.sample(cycle_ids, 1000)


In [44]:
# observe digital/analog names and some other
analog_set = set()
digital_set = set()
type_set = set(['GRAVITY', 'DART TEST', 'LEAK TEST', 'PREVAC', 'DART WARMUP'])
sample = []
for cycle in cycle_ids:
    cycle_data = dict(cf.fetch_cycle_json(str(int(cycle))))
    cycle_data = cycle_data['cycles'][0]
    cur_type = cycle_data['cycle_type']
    if cur_type in type_set:
        type_set.remove(cur_type)
        sample.append(cycle_data)
    if len(type_set) == 0:
        break
    # type_set.add(cycle_data["cycle_type"])
    # print(cycle, cycle_data["cycle_type"])
    # for digital in cycle_data["ioevents"]:
    #     digital_set.add(digital['name'])
    # analog = cycle_data['analog']
    # for ts in analog:
    #     for v in ts['values']:
    #         analog_set.add(v['channelCode'])


with open('sample.json', 'w') as f:
    json.dump(sample, f)


In [53]:
print(f"Cycle digital set: {sorted(['S43', 'NEUT2_SW', 'S09', 'S40', 'DOORB_PLK', 'NO_FLOOD', 'DOORASW2', 'DLS', 'DOORA_SEAL', 'DOORB_SEAL', 'NEUT1_SW', 'DOORA_CLSD', 'S35', 'S04', 'S37', 'S03', 'S02', 'S01', 'S07', 'DOORA_PLK'])}")
print(f"Analog channel set: {analog_set}")
print(f"Type set is {type_set}")

# Cycle digital set: {'S43', 'NEUT2_SW', 'S09', 'S40', 'DOORB_PLK', 'NO_FLOOD', 'DOORASW2', 'DLS', 'DOORA_SEAL', 'DOORB_SEAL', 'NEUT1_SW', 'DOORA_CLSD', 'S35', 'S04', 'S37', 'S03', 'S02', 'S01', 'S07', 'DOORA_PLK'}
# Analog channel set: {'RRTD', 'FXRES', 'GRTD', 'JRTD', 'TSENS', 'WRTD', 'CRTD', 'CPRES'}
# Type set is {'GRAVITY', 'DART TEST', 'LEAK TEST', 'PREVAC', 'DART WARMUP'}


Cycle digital set: ['DLS', 'DOORASW2', 'DOORA_CLSD', 'DOORA_PLK', 'DOORA_SEAL', 'DOORB_PLK', 'DOORB_SEAL', 'NEUT1_SW', 'NEUT2_SW', 'NO_FLOOD', 'S01', 'S02', 'S03', 'S04', 'S07', 'S09', 'S35', 'S37', 'S40', 'S43']
Analog channel set: set()
Type set is set()


In [19]:
# header the dataset .csv file that includes 98 features and 1 target label

features = ['cycle_type', 'cycle_duration', 
            
            'DLS_act_count', 'DOORASW2_act_count', 'DOORA_CLSD_act_count', 
            'DOORA_PLK_act_count', 'DOORA_SEAL_act_count', 
            'DOORB_PLK_act_count', 'DOORB_SEAL_act_count', 'NEUT1_SW_act_count', 'NEUT2_SW_act_count', 
            'NO_FLOOD_act_count', 'S01_act_count', 'S02_act_count', 'S03_act_count', 'S04_act_count', 
            'S07_act_count', 'S09_act_count', 'S35_act_count', 'S37_act_count', 'S40_act_count', 'S43_act_count',
 
            'DLS_init', 'DOORASW2_init', 'DOORA_CLSD_init', 'DOORA_PLK_init', 
            'DOORA_SEAL_init', 
            'DOORB_PLK_init', 'DOORB_SEAL_init', 'NEUT1_SW_init', 'NEUT2_SW_init', 
            'NO_FLOOD_init', 'S01_init', 'S02_init', 'S03_init', 'S04_init', 
            'S07_init', 'S09_init', 'S35_init', 'S37_init', 'S40_init', 
            'S43_init', 
            
            'DLS_end', 'DOORASW2_end', 'DOORA_CLSD_end', 'DOORA_PLK_end', 
            'DOORA_SEAL_end', 
            'DOORB_PLK_end', 'DOORB_SEAL_end', 'NEUT1_SW_end', 'NEUT2_SW_end', 
            'NO_FLOOD_end', 'S01_end', 'S02_end', 'S03_end', 'S04_end', 
            'S07_end', 'S09_end', 'S35_end', 'S37_end', 'S40_end', 
            'S43_end', 

            'DLS_first_act', 'DOORASW2_first_act', 'DOORA_CLSD_first_act', 'DOORA_PLK_first_act', 
            'DOORA_SEAL_first_act', 
            'DOORB_PLK_first_act', 'DOORB_SEAL_first_act', 'NEUT1_SW_first_act', 'NEUT2_SW_first_act', 
            'NO_FLOOD_first_act', 'S01_first_act', 'S02_first_act', 'S03_first_act', 'S04_first_act', 
            'S07_first_act', 'S09_first_act', 'S35_first_act', 'S37_first_act', 'S40_first_act', 
            'S43_first_act', 
            
            'RRTD_peak', 'FXRES_peak', 'GRTD_peak', 'JRTD_peak', 
            'TSENS_peak', 'WRTD_peak', 'CRTD_peak', 'CPRES_peak',

            'RRTD_pit', 'FXRES_pit', 'GRTD_pit', 'JRTD_pit', 
            'TSENS_pit', 'WRTD_pit', 'CRTD_pit', 'CPRES_pit',

            'FSR_Required']



In [61]:
# prepare the full dataset
with open('dataset3.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=features)
    counter = 0
    alarm_counter = 0
    missing_parts_counter = 0
    total = len(metadata_dict)
    for id, meta in metadata_dict.items():
        # if counter < 14421:
        #     counter += 1
        #     continue
        if id not in l:
            counter += 1
            continue
        duration = meta[2]
        if duration == 0:
            counter += 1
            continue
        target = meta[3]
        cycle = cf.fetch_cycle_json(str(int(id)))['cycles']
        if cycle == []:
            counter += 1
            continue
        cycle = cycle[0]
        if (cycle['has_missing_sections']):
            print('Cycle missing certain parts')
            missing_parts_counter += 1
            counter += 1
            continue
        print(f'Writing Cycle {id} to csv ...')
        cycle = Cycle(duration, cycle)
        init = cycle.digital_init()
        end = cycle.digital_end()
        act_count = cycle.act_count()
        first_act = cycle.first_act()
        peak, pit = cycle.analog_count_peak_pit()
        row = {}
        row['FSR_Required'] = target
        if (int(target) == 1):
            alarm_counter += 1
            print(f'registered {alarm_counter}th FSR cycle')
        row['cycle_duration'] = duration
        row['cycle_type'] = cycle.type
        digital_init = {x+'_init': y for x, y in init.items()}
        digital_end = {x+'_end': y for x, y in end.items()}
        digital_act_count = {x+'_act_count': y for x, y in act_count.items()}
        digital_first_act = {x+'_first_act': y for x, y in first_act.items()}
        analog_peak = {x+'_peak': y for x, y in peak.items()} 
        analog_pit = {x+'_pit': y for x, y in pit.items()}
        row.update(digital_init)
        row.update(digital_end)
        row.update(digital_act_count)
        row.update(digital_first_act)
        row.update(analog_peak)
        row.update(analog_pit)
        assert (len(init) == len(end) and len(end) == len(act_count) and len(act_count) == len(first_act)), 'digital feature lengths misalign'
        writer.writerow(row)
        counter += 1
        print(f'Writing complete. {counter}/{total} done.')
        # break
    print(f'Number of incomplete cycles: {missing_parts_counter}')



Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Cycle missing certain parts
Writing Cycle 14652039 to csv ...
registered 1th FSR cycle
Writing complete. 3245/15286 done.
Writing Cycle 14924290 to csv ...
registered 2th FSR cycle
Writing complete. 7737/15286 done.
Writing Cycle 14925133 to csv ...
registered 3th FSR cycle
Writing complete. 7738/15286 done.
Writing Cycle 14925828 to csv ...
registered 4th FSR cycle
Writing complete. 7739/15286 done.
Writing 

In [68]:
with open('dataset2.csv', 'r', newline='') as file:
    reader = csv.DictReader(file)
    c = 0
    for row in reader:
        c += 1
        # if reader[i][-1] == '1':
        #     print('alarm_found')
        #     c += 1
        if c == 10000:
            for key, value in row.items():
                if key != 'cycle_type':
                    row[key] = float(row[key])
            print(row)


{'cycle_type': 'PREVAC', 'cycle_duration': 3166.0, 'DLS_act_count': 1.0, 'DOORASW2_act_count': 0.0, 'DOORA_CLSD_act_count': 0.0, 'DOORA_PLK_act_count': 1.0, 'DOORA_SEAL_act_count': 1.0, 'DOORB_PLK_act_count': 0.0, 'DOORB_SEAL_act_count': 0.0, 'NEUT1_SW_act_count': 0.0, 'NEUT2_SW_act_count': 0.0, 'NO_FLOOD_act_count': 0.0, 'S01_act_count': 1.0, 'S02_act_count': 5.0, 'S03_act_count': 6.0, 'S04_act_count': 24.0, 'S07_act_count': 6.0, 'S09_act_count': 311.0, 'S35_act_count': 0.0, 'S37_act_count': 2.0, 'S40_act_count': 1.0, 'S43_act_count': 6.0, 'DLS_init': 0.0, 'DOORASW2_init': 0.0, 'DOORA_CLSD_init': 0.0, 'DOORA_PLK_init': 0.0, 'DOORA_SEAL_init': 0.0, 'DOORB_PLK_init': 0.0, 'DOORB_SEAL_init': 0.0, 'NEUT1_SW_init': 0.0, 'NEUT2_SW_init': 0.0, 'NO_FLOOD_init': 0.0, 'S01_init': 0.0, 'S02_init': 0.0, 'S03_init': 0.0, 'S04_init': 0.0, 'S07_init': 0.0, 'S09_init': 0.0, 'S35_init': 0.0, 'S37_init': 0.0, 'S40_init': 0.0, 'S43_init': 0.0, 'DLS_end': 1.0, 'DOORASW2_end': 0.0, 'DOORA_CLSD_end': 0.0, 